In [ ]:
from slackclient import SlackClient
from matplotlib.pyplot import pie, axis, show
import random
import os
import requests
import json
import time
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
#%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import warnings
from ibm_attrition_analysis import dict_return,add_charts,add_charts2

import io
from PIL import Image
from io import BytesIO

warnings.filterwarnings('ignore')

In [ ]:
slack_bot_token="---xxxxxxx---"
sc=SlackClient(slack_bot_token)
slack_token='---xxxxxxx---'
sc1=SlackClient(slack_token)

In [ ]:
#ML Model Classification Training

def mainfunc():
    random.seed(10)
    emp= pd.read_csv(r"D:\slackbot\env\Emp.csv")
    ## Data Pre-processing
    emp.Attrition.replace(('Yes', 'No'), (1, 0), inplace=True)
    businesstraveldummies = pd.get_dummies(emp['BusinessTravel'])
    DepartmentDummies = pd.get_dummies(emp['Department'])
    emp.EducationField=emp.EducationField.replace(['Human Resources'],'HumanResource')
    educationdummies = pd.get_dummies(emp['EducationField'])
    Genderdummies = pd.get_dummies(emp['Gender'])
    emp.JobRole=emp.JobRole.replace(['Human Resources'],'HumanResources')
    Jobroledummies= pd.get_dummies(emp['JobRole'])
    maritalstatusdummies= pd.get_dummies(emp['MaritalStatus'])
    emp = emp.join(businesstraveldummies)
    emp = emp.join(DepartmentDummies)
    emp = emp.join(educationdummies)
    emp = emp.join(Jobroledummies)
    emp = emp.join(maritalstatusdummies)
    emp = emp.join(Genderdummies)
    emp.OverTime.replace(('Yes', 'No'), (1, 0), inplace=True)
    emp.Over18.replace(('Y'),(1),inplace=True)
    ### Split Data
    ###split without dropping
    X = emp.drop('Attrition',axis=1)
    X.head()
    Y = emp['Attrition']
    Y.head()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=10)

    empcopy = X_test
    
    X =emp.drop(labels=['BusinessTravel','Department','EducationField','JobRole','MaritalStatus','Gender'],axis=1)
    #Y = emp['Attrition']
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=101)
    logmodel = LogisticRegression()
    X_train.dropna(inplace=True)
    y_train.dropna(inplace=True)
    logmodel.fit(X_train,y_train)
    X_test.dropna(inplace=True)
    y_test.dropna(inplace=True)
    prediction = logmodel.predict(X_test)
    pickle.dump(prediction,open('predictions.pkl','wb'))
    
    return X_train, X_test, y_train, y_test,empcopy,emp


In [ ]:
def getchannels():
    response=sc.api_call("users.list")
    for items in response.get('members'):
        print(items.get('id'))
def getmessage():
    messages=[]
    msg_history=requests.get('https://slack.com/api/channels.history?token=---xxxxxxx---&channel=---xxxxxxxyz---&pretty=1')
    json_response=msg_history.json()    
    json_tolist=json_response
    json_second_value=list(json_tolist.values())[1]
    #json_second_value=json_second_value[0]
    try:
        for elements in json_second_value:
            messages.append(elements.get('text'))
            return messages[0]
    except:
        return 0


In [ ]:
    
def postmessage(channel,text,attachment1):
    text=text
    attachment_new=[
        {
            "fallback": "DV Reports and Dashboards",
            "title": text,
            "title_link": "https://www.google.com",
            "text": "Take a look at it? ",
            "image_url": "https://res.cloudinary.com/jeff97/image/upload/v1543483253/DV1.png",
            "color": "#764FA5"
        }
    ]



    attachments_json = [
    {
        "fallback": "Select the Regions of Hiring",
        "color": "#3AA3E3",
        "actions": [
            {   "type": "button",
                "name": "Austin",
                "text": "Austin",
                "color": "danger",
                "style":"primary",
                "url":"http://<yourcloudvendor-dashboard-report-url>"
            },
            {   "type": "button",
                "name": "Bangalore",
                "text": "Bangalore",
                "color": "danger",
                "style":"primary",
                "url":"http://<yourcloudvendor-dashboard-report-url>"
            },
            {   "type": "button",
                "name": "Burlington",
                "text": "Burlington",
                "color": "danger",
                "style":"primary",
                "url":"http://<yourcloudvendor-dashboard-report-url>"
            },
            {   "type": "button",
                "name": "Reston",
                "text": "Reston",
                "color": "danger",
                "style":"primary",
                "url":"http://<yourcloudvendor-dashboard-report-url>"
            },
            {   "type": "button",
                "name": "Santa Monica",
                "text": "Santa Monica",
                "color": "danger",
                "style":"primary",
                "url":"http://<yourcloudvendor-dashboard-report-url>"
            }
        ]
    }
]
    attachment_DV=[
        {
            "fallback": "This is the attrition dashboard",
            "title": "This is the attrition dashboard",
            "title_link": "http://<yourcloudvendor-dashboard-report-url>",
            "text": "Attrition Analysis shows that bangalore hub has 32 terminations. We see that terminations are on a rise quarterly overall. It is seen that in the visual on the right, one of the main reason for termination is compensation.",
            "image_url": "https://res.cloudinary.com/jeff97/image/upload/v1559804352/slackdv.png",
            "color": "#764FA5"
}]
    if attachment1=='DV':
        response=sc.api_call(
            'chat.postMessage',
            channel=channel,
            text= "Please select your region of interest",
            attachments= attachments_json
        
            )
    if attachment1=='':
        response=sc.api_call(
            'chat.postMessage',
            channel=channel,
            text=text
        )
    if attachment1=='DV1':
        response=sc.api_call(
            'chat.postMessage',
            channel=channel,
            text="DV Dashboard",
            attachments=attachment_new
            )
    if attachment1=='ADB':
        response=sc.api_call(
            'chat.postMessage',
            channel=channel,
            attachments=attachment_DV
            )
    if attachment1=='drop':
        response=sc.api_call(
            'chat.postMessage',
            channel=channel,
            response_type="in_channel",
            text='test',
            attachments=[
    {
        "fallback": "Upgrade your Slack client to use messages like these.",
        "color": "#3AA3E3",
        "attachment_type": "default",
        "callback_id": "menu_options_2319",
        "actions": [
            {
                "name": "games_list",
                "text": "Pick a game...",
                "type": "select",
                "options":[
                    {"text":"Pick",
                     'value':'value1'},
                    {
                        'text':"pick1",
                        'value':'value2'
                        }
                    ]
            }
        ]
    }
]
            )
        print(response)

In [ ]:
def messagedelete(ts):
    response=sc1.api_call(
        'chat.delete',
        channel='<yourchannelid>',
        ts=ts
        )
    print(response)
    
def getmessage1():
    messages=[]
    msg_history=requests.get('https://slack.com/api/channels.history?token=---xxxxxxx---&channel=---xxxxxxxyz---&pretty=1')
    json_response=msg_history.json()
    json_tolist=json_response
    json_second_value=list(json_tolist.values())
    json_second_value=json_second_value[1]
    return json_second_value[0]['ts']

def getlatestmessagets():
    response=sc1.api_call(
        "channels.history",
        channel='<yourchannelid>',
        count=1
        )

    return response['messages'][0]['ts']


In [ ]:
def func1(X_train, X_test, y_train, y_test):
        ## LogisticRegression
    predictions=pickle.load(open('predictions.pkl','rb'))
    X_test["predictions"] = predictions
    X_test.head()
    X_test.rename(columns={'Male':'Gender'},inplace=True)
    sns_plot=sns.countplot(X_test['predictions'],data=X_test,hue='Gender')
    ax= plt.subplot()
    ax.set_xlabel('Female : Male                        Female  :  Male');
    ax.set_ylabel('Number of Employees'); 
    ax.set_title('Predicted Attrition by Gender'); 
    ax.xaxis.set_ticklabels(['Attrition:No', 'Attrition:Yes']);
    buf=io.BytesIO()
    #sns_plot.figure.savefig(r"D:\slackbot\env\Emp_Prediction_attr.png")
    sns_plot.figure.savefig(buf,format='png')
    plt.close()
    return buf

def show_and_uploadbuf(buf):
    #print(buf.getvalue())
    #buf.seek(0)
    my_file = {
                'file' : ('D:/gesture/Emp_Prediction_salover.png', buf, 'png')
               }
##    #buf.seek(0)
    #print(type(Image.open(buf)))
    #print("Thw type of image in buffer is",type(im))
    payload = {
                "filename":"myfile",
                "token":slack_token,
                "channels":['#virtualassistant']
               }
    r = requests.post("https://slack.com/api/files.upload", params=payload, files=my_file)
    #im.show()
    #buf.close()
    #plt.close()    

def func2(X_train, X_test, y_train, y_test):
    predicted_values=pickle.load(open('predictions.pkl','rb'))
    X_test["Predicted_values"] = predicted_values
    X_test.head()
    #################################################
    ## Attrition on Years of experience range ##
    #################################################
    bins = [0, 3, 6, 9, 12, 15, np.inf]
    names = ['0-2', '3-5', '6-8', '9-11', '12-14','>15']
    X_test['TotalworkingyrsRange'] = pd.cut(X_test['TotalWorkingYears'], bins, labels=names)
    X_test.rename(columns={'Male':'Gender','TotalworkingyrsRange':'Experience range'},inplace=True)
    sns_plot=sns.countplot(X_test['Predicted_values'],data=X_test,hue='Experience range')
    ax1= plt.subplot()
    #ax1.set_xlabel('Female : Male                        Female  :  Male');
    ax1.set_ylabel('Number of Employees'); 
    ax1.set_title('Predicted Attrition by Experience'); 
    ax1.xaxis.set_ticklabels(['Attrition:No', 'Attrition:Yes']); 
    buf=io.BytesIO()
    #sns_plot.figure.savefig(r"D:\slackbot\env\Emp_Prediction_attr.png")
    sns_plot.figure.savefig(buf,format='png')
    plt.close()
    return buf
def func3(X_train, X_test, y_train, y_test,empcopy):
    predictions=pickle.load(open('predictions.pkl','rb'))
    X_test["predictions"] = predictions
    empcopy["testpreds"] = predictions
    empcopy["ActualAttrition"] = y_test
    X_test.columns
    sums1 = empcopy.testpreds.groupby(empcopy.Department).sum()
    axis('equal')

    plt.pie(sums1,labels=['HR','Research&Development','Sales'],radius=1.25,autopct='%1.1f%%',shadow=True,labeldistance=1.05,colors=('Green','Orange','Red'));
    plt.title("Predicted Attrition by Dept",loc='right')
    #plt.savefig("D:/gesture/Emp_Prediction_deptnew.png")
    buf=io.BytesIO()
    #sns_plot.figure.savefig(r"D:\slackbot\env\Emp_Prediction_attr.png")
    plt.savefig(buf,format='png')
    plt.close()
    return buf

def func4(X_train, X_test, y_train, y_test,empcopy,emp):

    predictions=pickle.load(open('predictions.pkl','rb'))
    X_test["predictions"] = predictions
    empcopy["testpreds"] = predictions
    empcopy["ActualAttrition"] = y_test
    X_test.columns
    empcopy.columns

    binsMI = [0, 2000, 4000, 6000, 8000, 10000, 11000, np.inf]
    namesMI = ['0-1000', '2000-3000', '4000-5000', '5000-6000', '7000-8000', '9000-10000', '>10000']
    X_test['MontlyIncomeRange'] = pd.cut(emp['MonthlyIncome'], binsMI, labels=namesMI)
    f, axes = plt.subplots(1, 2)
    f.set_size_inches(15, 5)
    a=sns.countplot(X_test[X_test['predictions']==1].predictions,data=X_test,hue='MontlyIncomeRange',ax=axes[0],orient='v')
    b=sns.countplot(X_test[X_test['predictions']==1].predictions,data=X_test,hue='OverTime',ax=axes[1],orient='v')
    axes[0].set_xlabel('Predicted Attrition');
    axes[0].set_ylabel('Number of Employees');
    axes[0].set_title('Predicted Attrition vs Monthly Income');
    axes[1].set_xlabel('Predicted Attrition');
    axes[1].set_ylabel('Number of Employees');
    axes[1].set_title('Predicted Attrition vs Overtime');
    #a.figure.savefig("D:/gesture/Emp_Prediction_salover.png")
    buf=io.BytesIO()
    #sns_plot.figure.savefig(r"D:\slackbot\env\Emp_Prediction_attr.png")
    plt.savefig(buf,format='png')
    plt.close()
    return buf
    

In [ ]:

def upload(filepath):
##    my_file = {
##                'file' : (filepath, open(filepath, 'rb'), 'png')
#               }
    
    payload = {
                "filename":"myfile",
                "token":slack_token,
                "channels":['#virtualassistant']
               }
    r = requests.post("https://slack.com/api/files.upload", params=payload, files=buf)
    

def getlatestmessage():
    response=sc1.api_call(
        "channels.history",
        channel='<yourchannelid>',
        count=1
        
        )
    try:
        return response['messages'][0]['text']
    except KeyError:
        print(response)
    
def test(text,parameter):
    postmessage('<yourchannelid>',text,parameter)
    time.sleep(1)
def check_for_items(dictionary,text):
    for items in enumerate(dictionary):
        if items[1].lower() in text.lower():
            return items[1]
            break

if __name__=='__main__':
    print("Started")
    X_train, X_test, y_train, y_test,empcopy,emp=mainfunc()
    buf=func1(X_train, X_test, y_train, y_test)
    buf1=func2(X_train, X_test, y_train, y_test)
    buf2=func3(X_train, X_test, y_train, y_test,empcopy)
    buf3=func4(X_train, X_test, y_train, y_test,empcopy,emp)
    dict_mappings={"hey emma":["Hi! This is Emma your virtual assistant",''],"display attrition":["sharing",'DV'],'DV report':['This is the dashboard','ADB']}
    #ML_questions={"gender":buf,"department":buf2,"overtime":buf3,"experience":buf1}
    add_charts()
    add_charts2()
    ML_returned=dict_return()
    
    while(1):
        try:
            time.sleep(1)
            latest_message=getlatestmessage()
            
            if latest_message==None:
                latest_message=getlatestmessage()
        except:
            latest_message=getlatestmessage()
    
            
    
        #x=dict_mappings.get(latest_message)
        
        print(latest_message)
        #y=ML_questions.get(latest_message)
        x=check_for_items(dict_mappings,latest_message)
        x1=dict_mappings.get(x)
        y=check_for_items(ML_returned,latest_message)
        y1=ML_returned.get(y)
        print(x1)
        print(y1)
        
        if x1==None:
            #print("Sdsds")
            pass
        else:
            
            test(x1[0],x1[1])
            print("executed")
            time.sleep(1)
        if y1==None:
            pass
        else:
            show_and_uploadbuf(y1)
            time.sleep(1)
    
    
